# 스파스 벡터
- 정보 검색 및 자연어 처리에서 표면 수준의 용어 일치를 포착하는 중요한 방법
- 고밀도 벡터는 의미 이해에 탁월하지만, 희소 벡터는 특수 용어나 텍스트 식별자 검색에 일치 결과를 제공

## 개요
- 스파스 벡터는 대부분의 요소가 0이고 일부 차원만 0이 아닌 값을 갖는 특수한 고차원 벡터
- 스파스 벡터는 두 가지 접근 방식을 사용하여 생성할 수 있음
    - TF-IDF
    - 신경 스파스 임베딩 모델

참고자료: https://milvus.io/docs/ko/sparse_vector.md

## 데이터 형식

In [1]:
# 사전 목록{dimension_index: vlaue, ...} 형식
sparse_vectors = [{27: 0.5, 100: 0.3, 5369: 0.6}, {100: 0.1, 3: 0.8}]

In [ ]:
from scipy.sparse import csr_matrix

# First vector: indices [27, 100, 5369] with values [0.5, 0.3, 0.6]
# Second vector: indices [3, 100] with values [0.8, 0.1] 
indices = [[27, 100, 5369], [3, 100]]
values = [[0.5, 0.3, 0.6], [0.8, 0.1]]

sparse_vectors = [
    csr_matrix((vals, ([0]*len(idx), idx)), shape=(1, 5370))  # max index + 1
    for idx, vals in zip(indices, values)
]


In [6]:
# 튜플 이터러블 목록(예: [(dimension_index, value)])

sparse_vector = [
    (27, 0.5), (100, 0.3), (5369, 0.6),
    (100, 0.1), (3, 0.8)
]

## 컬렉션 스키마 정의
- 컬렉션을 만들기 전에 필드를 정의하는 컬렉션 스키마를 지정

### 필드 추가
- Milvus에서 스파스 벡터를 사용하려면 다음 필드를 포함하는 스키마로 컬렉션을 만들어야 함
    - VARCHAR 필드에서 자동 생성되거나 입력 데이터에서 직접 제공된 희소 벡터를 저장하기 위해 예약된 SPARSE_FLOAT_VECTOR 필드
    - 일반적으로 스파스 벡터가 나타내는 원시 텍스트도 컬렉션에 저장, VARCHAR 필드를 사용하여 원시 텍스트를 저장

In [18]:
from pymilvus import MilvusClient, DataType

client = MilvusClient(uri="http://localhost:19530")

schema = client.create_schema(
    auto_id=True,
    enable_dynamic_field=True,
)

schema.add_field(field_name="pk", datatype=DataType.VARCHAR, is_primary=True, max_length=100) # 최대 100바이트 길이로 자동 생성되는 VARCHAR 데이터 유형을 사용하여 기본 키 저장
schema.add_field(field_name="sparse_vector", datatype=DataType.SPARSE_FLOAT_VECTOR) # SPARSE_FLOAT_VECTOR 데이터 유형을 사용하여 SPARSE VECTOR 저장
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=65535, enable_analyzer=True) # VARCHAR 데이터 유형을 사용하여 텍스트 문자열 저장

{'auto_id': True, 'description': '', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 100}, 'is_primary': True, 'auto_id': False}, {'name': 'sparse_vector', 'description': '', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535, 'enable_analyzer': True}}], 'enable_dynamic_field': True}

# 인덱스 매개변수 설정
- 희소 벡터에 대한 인덱스를 생성하는 과정은 고밀도 벡터의 경우와 유사하지만 지정된 인덱스 유형(index_type), 거리 메트릭(metric_type), 인덱스 매개변수(params)에서 차이가 있음

In [19]:
index_params = client.prepare_index_params()

index_params.add_index(
    field_name="sparse_vector",
    index_name="sparse_inverted_index",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="IP",
    params={"inverted_index_algo": "DAAT_MAXSCORE"}, # or "DAAT_WAND" or "TAAT_NAIVE"
)


# 컬렉션 만들기

In [20]:
if client.has_collection("my_collection"):
    client.drop_collection("my_collection")

client.create_collection(
    collection_name="my_collection",
    schema=schema,
    index_params=index_params
)

# 데이터 삽입

In [21]:
data = [
    {
        "text": "information retrieval is a field of study.",
        "sparse_vector": {1: 0.5, 100: 0.3, 500: 0.8}
    },
    {
        "text": "information retrieval focuses on finding relevant information in large datasets.",
        "sparse_vector": {10: 0.1, 200: 0.7, 1000: 0.9}
    }
]

In [22]:
client.insert(
    collection_name="my_collection",
    data=data
)

{'insert_count': 2, 'ids': ['459127794345938952', '459127794345938953'], 'cost': 0}

# 유사도 검색  수행
- 스파스 벡터를 사용하여 유사도 검색을 수행하려면 쿼리 데이터와 검색 매개변수를 모두 준비

In [23]:
search_params = {
    "params": {"drop_ratio_search": 0.2},
}

query_data = [{1: 0.2, 50: 0.4, 1000: 0.7}]

In [24]:
res = client.search(
    collection_name="my_collection",
    data=query_data,
    limit=3,
    output_fields=["pk"],
    search_params=search_params,
)

In [25]:
print(res)

data: [[{'pk': '459127794345938953', 'distance': 0.6299999952316284, 'entity': {'pk': '459127794345938953'}}, {'pk': '459127794345938952', 'distance': 0.10000000149011612, 'entity': {'pk': '459127794345938952'}}]]
